In [1]:
import pandas as pd

fromngroups=2
tongroups=10

groups = {}

for g in range(fromngroups,tongroups+1):
    groups[g] = pd.read_csv(f"groups-{g}.csv")
    groups[g]['g'] = str(g)

In [2]:
groups[4]

,item,user-1,rating-1,user-2,rating-2,user-3,rating-3,user-4,rating-4,g
0,31,1024,3.0,129,4.0,663,NaN,138,3.0,4
1,23,1027,5.0,454,5.0,985,5.0,925,NaN,4
2,42,1313,4.0,1746,3.0,311,3.0,1613,NaN,4
3,51,788,5.0,186,4.0,1260,NaN,1711,5.0,4
4,169,48,5.0,199,NaN,840,3.0,399,5.0,4
...,...,...,...,...,...,...,...,...,...,...
9995,51,500,5.0,794,NaN,1278,4.0,319,4.0,4
9996,129,1395,NaN,390,5.0,1623,4.0,1165,5.0,4
9997,61,980,4.0,86,2.0,839,NaN,207,4.0,4
9998,100,1745,3.0,737,NaN,330,2.0,1006,4.0,4


In [3]:
#
# Matrix Factorization
#

pd_train = pd.read_csv('training-ratings.csv', header = 0, names=['u', 'i', 'r'])
pd_test = pd.read_csv('test-ratings.csv', header = 0, names=['u', 'i', 'r'])

from surprise import NMF
from surprise import accuracy
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(pd_train[['u', 'i', 'r']], reader)
trainset = data.build_full_trainset()

algo = NMF(biased=True)
algo.fit(trainset)

In [4]:
# predict
print(groups[4].tail(1))
algo.predict(19,129).est

      item  user-1  rating-1  user-2  rating-2  user-3  rating-3  user-4  \
9999   129      19       5.0     324       5.0       9       5.0    1406   

      rating-4  g  
9999       NaN  4  


4.441924443642847

In [5]:
groups[4][['user-1', 'item']].apply(lambda x: algo.predict(x[0], x[1]).est, axis=1)

0       3.690823
1       4.621954
2       4.416736
3       4.663624
4       4.628295
          ...   
9995    4.540526
9996    5.000000
9997    4.441101
9998    3.165221
9999    4.441924
Length: 10000, dtype: float64

In [6]:
for g in range(fromngroups,tongroups+1):
    for gi in range(1,g+1):
        groups[g]['pred-'+str(gi)] = groups[g][['user-'+str(gi), 'item']].apply(lambda x: algo.predict(x[0], x[1]).est, axis=1)

In [7]:
groups[4]

,item,user-1,rating-1,user-2,rating-2,user-3,rating-3,user-4,rating-4,g,pred-1,pred-2,pred-3,pred-4
0,31,1024,3.0,129,4.0,663,NaN,138,3.0,4,3.690823,3.424675,2.917237,3.290503
1,23,1027,5.0,454,5.0,985,5.0,925,NaN,4,4.621954,4.766854,4.628104,3.917434
2,42,1313,4.0,1746,3.0,311,3.0,1613,NaN,4,4.416736,3.405035,4.039774,3.756551
3,51,788,5.0,186,4.0,1260,NaN,1711,5.0,4,4.663624,4.328721,4.221263,3.996263
4,169,48,5.0,199,NaN,840,3.0,399,5.0,4,4.628295,4.595367,3.815219,4.390825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,51,500,5.0,794,NaN,1278,4.0,319,4.0,4,4.540526,4.492663,4.421846,4.108832
9996,129,1395,NaN,390,5.0,1623,4.0,1165,5.0,4,5.000000,4.208362,4.940428,5.000000
9997,61,980,4.0,86,2.0,839,NaN,207,4.0,4,4.441101,4.362601,4.276385,3.938617
9998,100,1745,3.0,737,NaN,330,2.0,1006,4.0,4,3.165221,3.157126,3.396629,3.854059


In [15]:
"""
INFORMATION
rating_mean -> media de los ratings
prediction_mean -> media de las predicciones
diff_rmean_pmean -> diff anteriores
indi_pred_error_<N> -> error en cada predicción.
indi_pred_emean -> MAE del modelo en test, cambia por conjuntos de test de cada grupo son distintos pero se mantiene en rangos con sentido
pred_error_<N> -> Diff (media de la predicción, modelo con el que comparar) con el rating individual
pred_emean -> Error de las predicciones
"""


for g in range(fromngroups,tongroups+1):
    groups[g]['rating_mean'] = groups[g].filter(regex='^rating-',axis=1).mean(axis=1, skipna=True)
    groups[g]['prediction_mean'] = groups[g].filter(regex='^pred-',axis=1).mean(axis=1, skipna=True)
    groups[g]['diff_rmean_pmean'] = abs(groups[g]['rating_mean']-groups[g]['prediction_mean']) 
    for gi in range(1,g+1):
        groups[g]['indi_pred_error_'+str(gi)] = abs(groups[g]['rating-'+str(gi)] - groups[g]['pred-'+str(gi)])
        groups[g]['indi_mean_error_'+str(gi)] = abs(groups[g]['rating-'+str(gi)] - groups[g]['rating_mean'])
        groups[g]['pred_error_'+str(gi)] = abs(groups[g]['rating-'+str(gi)] - groups[g]['prediction_mean'])
    groups[g]['indi_pred_emean'] = groups[g].filter(regex='^indi_pred_error_',axis=1).mean(axis=1, skipna=True)
    groups[g]['indi_mean_emean'] = groups[g].filter(regex='^indi_mean_error_',axis=1).mean(axis=1, skipna=True)
    groups[g]['pred_emean'] = groups[g].filter(regex='^pred_error_',axis=1).mean(axis=1, skipna=True)


In [16]:
# Cosas de mates...
from IPython.display import display, HTML
for g in range(fromngroups,tongroups+1):
    #print(g, groups[g]['diff'].describe()['mean'])
    print(g)
    display(HTML(groups[g].describe()[["rating_mean","prediction_mean", "diff_rmean_pmean", "indi_pred_emean", "indi_mean_emean", "pred_emean"]].T.to_html()))
    #display(HTML(groups[g].describe().T.to_html()))


2


,count,mean,std,min,25%,50%,75%,max
rating_mean,10000.0,3.798950,0.817264,1.0,3.500000,4.000000,4.500000,5.000000
prediction_mean,10000.0,3.776714,0.539206,1.0,3.459958,3.833409,4.165732,5.000000
diff_rmean_pmean,10000.0,0.509343,0.387641,0.0,0.205449,0.434600,0.727214,2.879839
indi_pred_emean,10000.0,0.706640,0.392574,0.0,0.421796,0.643835,0.926334,2.879839
indi_mean_emean,10000.0,0.492150,0.448089,0.0,0.000000,0.500000,0.500000,2.000000
pred_emean,10000.0,0.734239,0.402263,0.0,0.500000,0.606859,1.000000,2.879839


3


,count,mean,std,min,25%,50%,75%,max
rating_mean,10000.0,3.772183,0.819876,1.000000,3.333333,4.000000,4.500000,5.000000
prediction_mean,10000.0,3.767858,0.523000,1.291638,3.466975,3.824050,4.139946,4.918506
diff_rmean_pmean,10000.0,0.511269,0.388227,0.000034,0.204180,0.430934,0.731941,2.708730
indi_pred_emean,10000.0,0.709576,0.387239,0.000000,0.428985,0.650428,0.929913,2.845888
indi_mean_emean,10000.0,0.513928,0.446644,0.000000,0.000000,0.500000,0.722222,2.000000
pred_emean,10000.0,0.749392,0.399933,0.000563,0.500000,0.638339,1.000000,2.708730


4


...

,count,mean,std,min,25%,50%,75%,max
rating_mean,10000.0,4.061681,0.494754,1.600000,3.800000,4.166667,4.400000,5.000000
prediction_mean,10000.0,4.040136,0.347235,1.895880,3.809537,4.091245,4.305937,4.850910
diff_rmean_pmean,10000.0,0.303692,0.226752,0.000075,0.124479,0.260150,0.435436,1.529940
indi_pred_emean,10000.0,0.676380,0.236900,0.123158,0.507096,0.648351,0.817670,1.908395
indi_mean_emean,10000.0,0.639286,0.267840,0.000000,0.480000,0.640000,0.800000,1.833333
pred_emean,10000.0,0.715408,0.242413,0.005073,0.533830,0.679166,0.854767,1.930492


9


,count,mean,std,min,25%,50%,75%,max
rating_mean,10000.0,4.044510,0.501565,1.200000,3.800000,4.142857,4.400000,5.000000
prediction_mean,10000.0,4.025101,0.350648,1.911512,3.791166,4.076806,4.294318,4.834880
diff_rmean_pmean,10000.0,0.306330,0.234413,0.000050,0.124198,0.256637,0.439248,1.869509
indi_pred_emean,10000.0,0.679259,0.239719,0.086892,0.508567,0.650087,0.821640,1.991239
indi_mean_emean,10000.0,0.644408,0.270477,0.000000,0.480000,0.640000,0.800000,1.920000
pred_emean,10000.0,0.722710,0.247200,0.000195,0.538930,0.685181,0.862991,1.869509


10


,count,mean,std,min,25%,50%,75%,max
rating_mean,10000.0,4.087050,0.456909,1.666667,3.833333,4.142857,4.428571,5.000000
prediction_mean,10000.0,4.063188,0.320026,1.947527,3.842684,4.110492,4.308929,4.821657
diff_rmean_pmean,10000.0,0.281875,0.215146,0.000093,0.113537,0.238578,0.401318,1.696805
indi_pred_emean,10000.0,0.675781,0.217943,0.071946,0.520591,0.654561,0.805215,1.829343
indi_mean_emean,10000.0,0.648581,0.246654,0.000000,0.489796,0.653061,0.816327,1.781250
pred_emean,10000.0,0.714942,0.225454,0.002730,0.549940,0.675221,0.839130,1.987514


In [12]:
# Cosas de mates...
from IPython.display import display, HTML
for g in range(fromngroups,tongroups+1):
    #print(g, groups[g]['diff'].describe()['mean'])
    print(g)
    #display(HTML(groups[g].describe()[["rating_mean","prediction_mean", "diff_rmean_pmean", "indi_pred_emean", "pred_emean"]].T.to_html()))
    display(HTML(groups[g].describe().T.to_html()))

2


,count,mean,std,min,25%,50%,75%,max
item,10000.0,144.323300,141.121792,0.0,38.000000,105.500000,208.000000,873.000000
user-1,10000.0,873.710500,503.629398,0.0,448.000000,883.000000,1296.000000,1757.000000
rating-1,10000.0,3.794600,1.055425,1.0,3.000000,4.000000,5.000000,5.000000
user-2,10000.0,873.356300,497.810834,3.0,451.000000,874.000000,1291.000000,1757.000000
rating-2,10000.0,3.803300,1.052578,1.0,3.000000,4.000000,5.000000,5.000000
pred-1,10000.0,3.787086,0.603030,1.0,3.429774,3.843292,4.208773,5.000000
pred-2,10000.0,3.766342,0.603528,1.0,3.413531,3.823650,4.182226,5.000000
rating_mean,10000.0,3.798950,0.817264,1.0,3.500000,4.000000,4.500000,5.000000
prediction_mean,10000.0,3.776714,0.539206,1.0,3.459958,3.833409,4.165732,5.000000
diff_rmean_pmean,10000.0,0.509343,0.387641,0.0,0.205449,0.434600,0.727214,2.879839


3


,count,mean,std,min,25%,50%,75%,max
item,10000.0,153.116100,144.951538,0.000000,43.000000,113.000000,214.000000,988.000000
user-1,10000.0,867.904000,509.863587,0.000000,419.000000,868.500000,1301.000000,1756.000000
rating-1,6846.0,3.793456,1.064601,1.000000,3.000000,4.000000,5.000000,5.000000
user-2,10000.0,866.450600,500.460195,0.000000,436.000000,868.500000,1280.500000,1757.000000
rating-2,7150.0,3.783776,1.065863,1.000000,3.000000,4.000000,5.000000,5.000000
user-3,10000.0,878.472400,500.779120,2.000000,447.000000,874.000000,1307.000000,1757.000000
rating-3,6999.0,3.768395,1.073570,1.000000,3.000000,4.000000,5.000000,5.000000
pred-1,10000.0,3.781048,0.605531,1.000000,3.422588,3.835670,4.206431,5.000000
pred-2,10000.0,3.771644,0.609128,1.000000,3.413366,3.823898,4.194870,5.000000
pred-3,10000.0,3.750881,0.606508,1.077975,3.397255,3.802199,4.175847,5.000000


4


...

,count,mean,std,min,25%,50%,75%,max
item,10000.0,69.697400,75.133667,0.000000,16.000000,43.000000,86.000000,661.000000
user-1,10000.0,877.610500,513.460693,0.000000,433.000000,896.000000,1315.000000,1753.000000
rating-1,6489.0,4.088920,0.923365,1.000000,4.000000,4.000000,5.000000,5.000000
user-2,10000.0,867.523300,505.187341,0.000000,434.000000,865.500000,1300.000000,1754.000000
rating-2,6635.0,4.081085,0.936190,1.000000,4.000000,4.000000,5.000000,5.000000
user-3,10000.0,860.038800,505.723981,0.000000,420.000000,853.000000,1289.000000,1753.000000
rating-3,6730.0,4.079049,0.949298,1.000000,4.000000,4.000000,5.000000,5.000000
user-4,10000.0,882.990900,503.659486,3.000000,456.000000,887.000000,1304.000000,1757.000000
rating-4,6727.0,4.070611,0.936860,1.000000,4.000000,4.000000,5.000000,5.000000
user-5,10000.0,884.128600,505.154445,3.000000,455.750000,888.000000,1318.250000,1757.000000


9


,count,mean,std,min,25%,50%,75%,max
item,10000.0,73.894300,77.257876,0.000000,16.000000,44.000000,113.000000,742.000000
user-1,10000.0,884.166900,508.484447,0.000000,448.000000,896.000000,1314.000000,1752.000000
rating-1,5864.0,4.064973,0.935425,1.000000,4.000000,4.000000,5.000000,5.000000
user-2,10000.0,873.647300,510.379405,0.000000,435.000000,882.000000,1313.000000,1752.000000
rating-2,5930.0,4.068297,0.960103,1.000000,3.000000,4.000000,5.000000,5.000000
user-3,10000.0,870.421800,507.162694,0.000000,433.750000,866.000000,1302.000000,1754.000000
rating-3,6053.0,4.084421,0.939148,1.000000,4.000000,4.000000,5.000000,5.000000
user-4,10000.0,879.147200,509.911180,1.000000,439.000000,874.000000,1318.000000,1756.000000
rating-4,6084.0,4.060651,0.949542,1.000000,4.000000,4.000000,5.000000,5.000000
user-5,10000.0,873.891900,501.665475,3.000000,454.000000,871.000000,1303.000000,1757.000000


10


,count,mean,std,min,25%,50%,75%,max
item,10000.0,62.194400,66.790319,0.000000,16.000000,43.000000,81.000000,571.000000
user-1,10000.0,875.797600,513.242448,0.000000,432.000000,884.000000,1313.000000,1748.000000
rating-1,6218.0,4.101158,0.923604,1.000000,4.000000,4.000000,5.000000,5.000000
user-2,10000.0,872.384500,511.773306,0.000000,433.000000,870.000000,1313.000000,1751.000000
rating-2,6254.0,4.110169,0.926802,1.000000,4.000000,4.000000,5.000000,5.000000
user-3,10000.0,867.685100,508.110759,0.000000,421.000000,864.500000,1300.000000,1754.000000
rating-3,6446.0,4.114024,0.938215,1.000000,4.000000,4.000000,5.000000,5.000000
user-4,10000.0,872.956300,506.130221,1.000000,435.000000,868.000000,1312.250000,1756.000000
rating-4,6443.0,4.109576,0.939545,1.000000,4.000000,4.000000,5.000000,5.000000
user-5,10000.0,866.620200,502.094679,1.000000,435.000000,857.000000,1299.250000,1757.000000
